In [ ]:
# Transformer 다국어 기계 번역 모델 & 파인 튜닝
# - Hugging Face 라이브러리 적용
# - AI HUB 금융 다국어 번역 데이터셋 적용
# - 입력된 문장을 다국어 기계 번역 모델을 통한 영어->한국어, 한국어->번역
# 1. 학습 목표
# - 구조 최적화 및 파이프라인 단순화
# - AI HUB 방송 다국어 번역 데이터셋 전처리
# - 병렬 문장쌍 데이터셋 변환 전처리
# - 토크나이징 및 토크나이징 전처리
# - 베이스 모델 로드
# - LoRA(Low-Rank Adaptation) 설정, 특정 레이어에 작은 저차원 행렬(랭크 r)을 삽입해서 학습
# - LoRA(Low-Rank Adaptation) 모델, 메모리 효율성/빠른 학습/도메인 적용, base 모델에 여러 LoRA 모듈을 붙였다 떼었다 할 수 있음
# - 학습 args 설정
# - Trainer 정의
# - Trainer 실행
# - LoRA 적용된 모델 저장, LoRA모델/토크나이저
# - LoRA 적용된 모델 불러오기, 베이스모델/LoRA모델/토크나이저

In [1]:
import torch
import numpy as np
import glob, json, re, os, random, csv

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__, device)

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
print("빌드 정보:", torch.__version__)
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))

2.8.0+cu129 cuda
CUDA 사용 가능 여부: True
PyTorch CUDA 버전: 12.9
빌드 정보: 2.8.0+cu129
사용 중인 GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [30]:
# 데이터셋 전처리 - AI HUB 방송 다국어 번역 데이터셋
ko_lines, en_lines = [], []
folders = [ # 폴더 리스트 정의
    './llm_data/ai_hub_article/*.json'
]

# 모든 JSON 읽기
for folder in folders:
    for path in glob.glob(folder): # 특정 디렉토리에서 지정한 패턴과 일치하는 모든 파일 경로를 리스트로 반환
        with open(path, encoding='utf-8') as f:
            try:
                data = json.load(f) # 파일 전체 로드(dict 구조)
            except json.JSONDecodeError:
                continue

            # sents 리스트에서 문장쌍 추출
            for sent in data.get('sents', []):
                en = sent.get('mtpe') # 원문(영어)
                ko = sent.get('source_cleaned') # 최종번역문(한국어) 추출

                if en and ko and ko != 'N/A':
                    en_lines.append(en.strip())
                    ko_lines.append(ko.strip())

# 1. Detokenize 함수 정의
def detokenize_sentence(sentence: str) -> str:
    sentence = sentence.strip()
    sentence = re.sub(r"\s+([?.!,])", r"\1", sentence)  # " ?" → "?"
    sentence = re.sub(r"\s+", " ", sentence)            # 여러 공백 → 하나
    return sentence

# 2. 데이터셋 전처리
en_lines = [detokenize_sentence(s) for s in en_lines]
ko_lines = [detokenize_sentence(s) for s in ko_lines]


print(f'총 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')
print(ko_lines[0])
print(en_lines[0])

총 문장쌍 개수: 80883, 80883
2019년부터 전년말 기준 자산총액 2조 원 이상인 기업에 대해 의무화하였으며, 2020년부터는 전년말 기준 자산총액 5천 억 원 이상 기업까지 의무대상을 확대했다.
From 2019, it was mandatory for companies with total assets of KRW 2 trillion or more as of the end of the previous year, and from 2020, the mandatory target was expanded to companies with total assets of KRW 500 billion or more as of the end of the previous year.


In [31]:
# 데이터 전처리 - 중복 제거 및 순서 유지
# pairs = list(set(zip(en_lines, ko_lines)))
# en_lines, ko_lines = zip(*pairs) # 다시 분리
seen = set()
pairs = []
for en, ko in zip(en_lines, ko_lines):
    if (en, ko) not in seen:
        # 새로운 문장쌍을 집합에 기록, 이후 같은 문장쌍이 나오면 if 조건에서 걸러져 추가되지 않는다
        seen.add( (en, ko) )
        
        # 중복이 아닌 문장쌍을 리스트에 추가, 원래 순서대로 중복 없는 문장쌍 리스트가 만들어 진다
        # - pairs는 [("Hello","안녕"), ("Goodbye","잘가")] 
        pairs.append( (en, ko) )

# 이를 다시 분리 - 영어 문장들만 모아 ("Hello","Goodbye"), 한국어 문장들만 모아 ("안녕","잘가")
en_lines, ko_lines = zip(*pairs)

print(f'중복 제거 후 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')
print(ko_lines[0])
print(en_lines[0])

중복 제거 후 문장쌍 개수: 80883, 80883
2019년부터 전년말 기준 자산총액 2조 원 이상인 기업에 대해 의무화하였으며, 2020년부터는 전년말 기준 자산총액 5천 억 원 이상 기업까지 의무대상을 확대했다.
From 2019, it was mandatory for companies with total assets of KRW 2 trillion or more as of the end of the previous year, and from 2020, the mandatory target was expanded to companies with total assets of KRW 500 billion or more as of the end of the previous year.


In [32]:
# 데이터 전처리 - 샘플링 추가

# 샘플링 최대 50,000 문장만 사용
# sample_size = 50000
sample_size = len(en_lines)
if len(ko_lines) > sample_size:
    indices = random.sample(range(len(ko_lines)), sample_size)
    ko_lines = [ ko_lines[i] for i in indices ]
    en_lines = [ en_lines[i] for i in indices ]

print(f'샘플링 후 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')
print(ko_lines[0])
print(en_lines[0])

샘플링 후 문장쌍 개수: 80883, 80883
2019년부터 전년말 기준 자산총액 2조 원 이상인 기업에 대해 의무화하였으며, 2020년부터는 전년말 기준 자산총액 5천 억 원 이상 기업까지 의무대상을 확대했다.
From 2019, it was mandatory for companies with total assets of KRW 2 trillion or more as of the end of the previous year, and from 2020, the mandatory target was expanded to companies with total assets of KRW 500 billion or more as of the end of the previous year.


In [33]:
# 데이터 전처리 - 저장
out_dir = './llm_data/ai_hub_article_translation'

# 폴더 없을시 생성
if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)
    print(f'폴더 생성 완료: {out_dir}')
else:
    print(f'이미 존재하는 폴더: {out_dir}')

ko_path = f'{out_dir}/train_ko.txt'
en_path = f'{out_dir}/train_en.txt'

with open(ko_path, 'w', encoding='utf-8') as fko, \
    open(en_path, 'w', encoding='utf-8') as fen:
    for k, e in zip(ko_lines, en_lines):
        fko.write(k + '\n')
        fen.write(e + '\n')
print('저장 완료', ko_path, en_path)

이미 존재하는 폴더: ./llm_data/ai_hub_article_translation
저장 완료 ./llm_data/ai_hub_article_translation/train_ko.txt ./llm_data/ai_hub_article_translation/train_en.txt


In [34]:
# AI Hub 방송 데이터셋(train_ko.txt, train_en.txt) -> CSV로 변환해 파인 튜닝
# 원본 데이터는 train_ko.txt, train_en.txt로 분리 -> 병렬 문장쌍을 만들어야 함
# 파인튜닝시 양방향 번역을 지원하려면 같은 문장쌍은 en->ko, ko->en 두방향으로 모두 포함해야 함
# CSV 구조 예시
# src,tgt,src_lang,tgt_lang
# You can buy it from a convenience store try it out.,편의점에서 사실 수 있으니 시도해보시길 바랍니다.,en,ko
# 편의점에서 사실 수 있으니 시도해보시길 바랍니다.,You can buy it from a convenience store try it out.,ko,en
# She frees him and takes him as her navigator.,그녀는 그를 풀어주고 조종자로써 그를 데려간다.,en,ko
# 그녀는 그를 풀어주고 조종자로써 그를 데려간다.,She frees him and takes him as her navigator.,ko,en
# Iyengar's belief in Gandhi's philosophy is so deep that he can't even open his laptop without remorse.,간디의 철학에 대한 아이옌가르의 믿음은 너무 깊어서, 그는 심지어 그의 노트북도 양심의 가책 없이 열 수 없다.,en,ko
# 간디의 철학에 대한 아이옌가르의 믿음은 너무 깊어서, 그는 심지어 그의 노트북도 양심의 가책 없이 열 수 없다.,Iyengar's belief in Gandhi's philosophy is so deep that he can't even open his laptop without remorse.,ko,en
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model

# 전체 데이터 로드 - AI Hub 방송 데이터셋(train_ko.txt, train_en.txt)
with open('./llm_data/ai_hub_article_translation/train_en.txt', 'r', encoding='utf-8') as f_en, \
    open('./llm_data/ai_hub_article_translation/train_ko.txt', 'r', encoding='utf-8') as f_ko:
    en_lines = f_en.read().splitlines()
    ko_lines = f_ko.read().splitlines()

# 데이터 개수 제한 (예: 100개)
limit = 1000
en_lines = en_lines[:limit]
ko_lines = ko_lines[:limit]

# train/valid split(90 : 10)
split_idx = int(len(en_lines) * 0.9)
train_en, valid_en = en_lines[:split_idx], en_lines[split_idx:]
train_ko, valid_ko = ko_lines[:split_idx], ko_lines[split_idx:]

print(len(train_en))
print(len(valid_en))

900
100


In [35]:
# 병렬 데이터 생성 - 편향되지 않은 데이터셋 생성
# 영어->한국어, 한국어->영어

# train.csv 생성
with open('./llm_data/ai_hub_article_translation/train.csv', 'w', encoding='utf-8', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(['src', 'tgt', 'src_lang', 'tgt_lang'])

    print(train_en[0])
    print(train_ko[0])
    for en, ko in zip(train_en, train_ko):
        en, ko = en.strip(), ko.strip()
        if not en or not ko:
            continue

        # 영어 -> 한국어
        writer.writerow([en, ko, 'en', 'ko'])
        # 한국어 -> 영어
        writer.writerow([ko, en, 'ko', 'en'])

# valid.csv 생성
with open('./llm_data/ai_hub_article_translation/valid.csv', 'w', encoding='utf-8', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(['src', 'tgt', 'src_lang', 'tgt_lang'])

    for en, ko in zip(valid_en, valid_ko):
        en, ko = en.strip(), ko.strip()
        if not en or not ko:
            continue
        
        # 영어 -> 한국어
        writer.writerow([en, ko, 'en', 'ko'])
        # 한국어 -> 영어
        writer.writerow([ko, en, 'ko', 'en'])

From 2019, it was mandatory for companies with total assets of KRW 2 trillion or more as of the end of the previous year, and from 2020, the mandatory target was expanded to companies with total assets of KRW 500 billion or more as of the end of the previous year.
2019년부터 전년말 기준 자산총액 2조 원 이상인 기업에 대해 의무화하였으며, 2020년부터는 전년말 기준 자산총액 5천 억 원 이상 기업까지 의무대상을 확대했다.


In [19]:
# 토크나이저 전처리
from datasets import load_dataset # Hugging Face의 데이터셋 관리 라이브러리(학습용 데이터 로딩에 사용)
# M2M100 모델과 토크나이저, 학습 관련 유틸리티 제공
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model # LoRA 설정을 위한 라이브러리(모델 파라미터 효율적 파인튜닝)

# 1. tokenizer 로드, Hugging Face M2M100-418M 모델의 토크나이저 로드
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 2. 전처리 함수 (동적 tgt_lang 설정)
def preprocess_function(examples):
    # 데이터셋에서 src(원문), tgt(번역문), tgt_lang(목표 언어 코드) 가져옴
    inputs = examples["src"]
    targets = examples["tgt"]
    tgt_langs = examples["tgt_lang"]

    model_inputs = tokenizer(
        inputs, # src(원문)을 토큰화
        max_length=128,  # 최대 길이 128
        truncation=True, # 최대 길이 초과시 잘라냄
        padding="max_length" # 길이가 부족하면 padding 처리
    )

    labels_list = []
    for text, lang in zip(targets, tgt_langs):
        # 각 문장마디 목표 언어(tgt_lang)
        if lang in tokenizer.lang_code_to_id:
            tokenizer.tgt_lang = lang
        else:
            tokenizer.tgt_lang = "en"  # 기본값
        
        # 타겟 문장 토큰화
        with tokenizer.as_target_tokenizer(): # 번역 대상 문장을 토큰화할때 사용하는 모드
            labels = tokenizer(
                text, # tgt(번역문)
                max_length=128, 
                truncation=True, 
                padding="max_length"
            )
        
        # labels["input_ids"]를 추출해서 학습용 정답(label)로 저장
        labels_list.append(labels["input_ids"])

    # 입력(src)와 정답(tgt)을 모두 포함한 딕셔너리 반환, Trainer가 이 반환값을 받아서 학습에 사용
    # {
    #     "input_ids": [...],        # 원문 토큰 시퀀스
    #     "attention_mask": [...],   # 패딩 여부 표시
    #     "labels": [...]            # 번역문 토큰 시퀀스
    # }
    model_inputs["labels"] = labels_list
    return model_inputs

In [20]:
# 데이터셋 불러오기
dataset = load_dataset( # Hugging Face datasets 라이브러리를 사용해 CSV 파일을 로드
    "csv",
    data_files={
        "train": "./llm_data/ai_hub_article_translation/train.csv",
        "validation": "./llm_data/ai_hub_article_translation/valid.csv"
    }
)

# 토크나이즈 적용
# - 결과: 각 샘플이 {"input_ids": ..., "attention_mask": ..., "labels": ...} 
tokenized_dataset = dataset.map( # map() 함수로 앞서 정의한 preprocess_function을 데이터셋 전체에 적용
    preprocess_function, 
    batched=True # 여러 샘플을 한번에 처리하여 속도 향상
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [21]:
# 모델 로드

# pretrained model M2M100_418M 로드, 베이스 모델로 사용
base_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

# LoRA 설정
lora_config = LoraConfig(
    r=8,              # 랭크 크기(저차원 행렬 크기), 작을수록 가볍고 빠르지만 표현력이 줄어듬
    lora_alpha=32,    # 스케일링 계수, 학습된 LoRA 행렬을 원래 모델에 얼마나 반영할지 결정하는 스케일
    lora_dropout=0.1, # 드롭아웃
    target_modules=["q_proj", "v_proj"]  # Attention 모듈에 적용, Query/Value 저차원 행렬만 학습
)

# LoRA 모델 생성, PEFT(Param-Efficient Fine-Tuning) 모델 생성
model = get_peft_model(base_model, lora_config) # 전체 모델 파라미터는 그대로 두고 LoRA 모듈만 학습 대상이 됨

In [22]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="./llm_models/results_lora",    # 학습 결과(모델, 체크포인트) 저장 경로
    eval_strategy="epoch",          # 구버전, 매 epoch마다 평가
    learning_rate=5e-5,             # 학습률
    per_device_train_batch_size=16, # 학습 배치 크기
    per_device_eval_batch_size=16,  # 평가 배치 크기
    num_train_epochs=3,             # 학습 epoch 수
    weight_decay=0.01,              # 가중치 감쇠(정규화)
    save_total_limit=2,             # 체크포인트 최대 저장 개수
    logging_dir="./results_lora_logs",           # 로그 저장 경로
    logging_steps=100,              # 100 step마다 로그 기록
)

# Trainer 정의
trainer = Trainer(
    model=model,        # LoRA 적용된 모델
    args=training_args, # 학습 설정
    train_dataset=tokenized_dataset["train"],       # 학습 데이터셋
    eval_dataset=tokenized_dataset["validation"],   # 평가 데이터셋
)

In [23]:
# 학습 실행
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [13]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 영어 → 한국어
text = "It is literally the basis of life."
inputs = tokenizer(text, return_tensors="pt").to(device)   # 입력도 GPU로 이동
forced_bos_token_id = tokenizer.lang_code_to_id["ko"]
outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id)
print("EN→KO:", tokenizer.decode(outputs[0], skip_special_tokens=True))

# 한국어 → 영어
text = "말 그대로 삶의 기초입니다."
inputs = tokenizer(text, return_tensors="pt").to(device)   # 입력도 GPU로 이동
forced_bos_token_id = tokenizer.lang_code_to_id["en"]
outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id)
print("KO→EN:", tokenizer.decode(outputs[0], skip_special_tokens=True))

EN→KO: It is literally the basis of life.
KO→EN: 말 그대로 삶의 기초이다.


In [14]:
# LoRA 적용된 모델 저장
model.save_pretrained("./llm_models/lora_translation_finance_model")
tokenizer.save_pretrained("./llm_models/lora_translation_finance_model")

('./llm_models/lora_translation_finance_model\\tokenizer_config.json',
 './llm_models/lora_translation_finance_model\\special_tokens_map.json',
 'llm_models\\lora_translation_finance_model\\vocab.json',
 'llm_models\\lora_translation_finance_model\\sentencepiece.bpe.model',
 './llm_models/lora_translation_finance_model\\added_tokens.json')

In [15]:
from peft import PeftModel

# 원본 M2M100 모델 로드
base_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

# LoRA 어댑터 붙여서 불러오기
model = PeftModel.from_pretrained(base_model, "./llm_models/lora_translation_finance_model")

# 토크나이저도 불러오기
tokenizer = M2M100Tokenizer.from_pretrained("./llm_models/lora_translation_finance_model")

# 디바이스 맞추기
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [16]:
# 영어 → 한국어
text = "It is literally the basis of life."
texts = [
    "The weather is very good today.",
    "I am studying machine learning.",
    "Artificial intelligence is changing the world.",
    "Deep learning is a subset of machine learning.",
    "Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day.",
    "When women replaced men, they earned much less money than them, which was a real problem.",
    "It is literally the basis of life.",
    "It's hard to believe, but the People's Artist of Russia, singer and TV presenter Nadezhda Babkina is 70 years old!",
    "Arriving in the USA, Nuno is assigned the teacher Jane Dayle Haddon as a &quot;tutor&quot;.",
    "This mediated dialogue allows Stefania and Andrea to reflect on their relationship and, when the game is revealed, to resume it with greater awareness.",
    "The findings of this analysis, as demonstrated during the recent global financial crisis, indicate that the impact of volatility in overseas financial markets on the domestic financial market has undergone long-term and continuous changes, reflecting the increased interconnectivity between domestic and foreign financial markets since the foreign exchange crisis."

]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["ko"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("EN→KO:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("EN→KO:")
for i, output in enumerate(outputs):
    print(f"{texts[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


# 한국어 → 영어
text = "말 그대로 삶의 기초입니다."
texts_ko = [
    "오늘 날씨는 매우 좋습니다.",
    "저는 머신러닝을 공부하고 있습니다.",
    "인공 지능은 세계를 변화시킵니다.",
    "딥러닝은 머신 러닝의 하위 세트입니다.",
    "지엠 판매원인 로베르토는 그가 매일 편지를 쓰는 오르넬라 무티를 열렬히 사랑한다.",
    "여성이 남성을 대체했을 때, 그들은 남성보다 훨씬 적은 돈을 벌게 되었고 이것이 진짜 문제였습니다.",
    "말 그대로 삶의 기초입니다.",
    "믿기 어렵겠지만, 러시아의 인민 예술가이자 가수이자 텔레비전 진행자인 나데즈다 밥키나는 70세입니다!",
    "미국에 도착한 누노는 제인 데일 해든 선생님으로 임명되었다.",
    "이 중재된 대화를 통해 스테파니아와 안드레아는 그들의 관계에 대해 반성하고 게임이 공개되면 더 큰 인식으로 게임을 재개할 수 있습니다.",
    "이러한 분석 결과는 최근의 글로벌 금융위기 기간에 잘 드러났듯이 국내외 금융 시장 간 연계성이 외환위기 이후 커진 가운데 해외 금융시장의 변동성이 국내 금융 시장에 미치는 파급효과도 보다 장기적이고 지속적으로 변화된 것을 의미한다."

]
inputs = tokenizer(texts_ko, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["en"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("KO→EN:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("\nKO→EN:")
for i, output in enumerate(outputs):
    print(f"{texts_ko[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


EN→KO:
The weather is very good today. → The weather is very good today.
I am studying machine learning. → I am studying machine learning.
Artificial intelligence is changing the world. → Artificial Intelligence is changing the world.
Deep learning is a subset of machine learning. → Deep learning is a subset of machine learning.
Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day. → Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day.
When women replaced men, they earned much less money than them, which was a real problem. → When women replaced men, they earned much less money than themselves, which was a real problem.
It is literally the basis of life. → It is literally the basis of life.
It's hard to believe, but the People's Artist of Russia, singer and TV presenter Nadezhda Babkina is 70 years old! → It’s hard to believe, but the People’s Artist of Russia, singer and TV presentator Nadezhda Babkina is 70 years 

In [ ]:
# 번역 :      이러한 분석 결과는 최근의 글로벌 금융위기 기간에 잘 드러났듯이 국내외 금융 시장 간 연계성이 외환위기 이후 커진 가운데 해외 금융시장의 변동성이 국내 금융 시장에 미치는 파급효과도 보다 장기적이고 지속적으로 변화된 것을 의미한다.
# 파인튜닝 :   최근의 글로벌 금융 위기에서 증명된 이 분석의 결과에 따르면 해외 금융 시장의 변동성의 국내 금융 시장에 미치는 영향은 외환 위기 이후 국내 금융 시장과 외국 금융 시장 간의 상호 연결성 증가를 반영하는 장기적 및 지속적인 변화를 경험했습니다.
# 베이스모델 : 최근의 글로벌 금융 위기에서 보여진 이 분석의 결과에 따르면 국내 금융 시장에 대한 해외 금융 시장의 변동성의 영향은 장기적이고 지속적인 변화를 겪었으며, 이는 외환 위기 이후 국내 금융 시장과 외국 금융 시장 간의 상호 연결이 증가한 것을 반영한다.

# 베이스 모델 테스트
import torch

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 영어 → 한국어
text = "It is literally the basis of life."
texts = [
    "The weather is very good today.",
    "I am studying machine learning.",
    "Artificial intelligence is changing the world.",
    "Deep learning is a subset of machine learning.",
    "Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day.",
    "When women replaced men, they earned much less money than them, which was a real problem.",
    "It is literally the basis of life.",
    "It's hard to believe, but the People's Artist of Russia, singer and TV presenter Nadezhda Babkina is 70 years old!",
    "Arriving in the USA, Nuno is assigned the teacher Jane Dayle Haddon as a &quot;tutor&quot;.",
    "This mediated dialogue allows Stefania and Andrea to reflect on their relationship and, when the game is revealed, to resume it with greater awareness.",
    "The findings of this analysis, as demonstrated during the recent global financial crisis, indicate that the impact of volatility in overseas financial markets on the domestic financial market has undergone long-term and continuous changes, reflecting the increased interconnectivity between domestic and foreign financial markets since the foreign exchange crisis."

]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["ko"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("EN→KO:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("EN→KO:")
for i, output in enumerate(outputs):
    print(f"{texts[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


# 한국어 → 영어
text = "말 그대로 삶의 기초입니다."
texts_ko = [
    "오늘 날씨는 매우 좋습니다.",
    "저는 머신러닝을 공부하고 있습니다.",
    "인공 지능은 세계를 변화시킵니다.",
    "딥러닝은 머신 러닝의 하위 세트입니다.",
    "지엠 판매원인 로베르토는 그가 매일 편지를 쓰는 오르넬라 무티를 열렬히 사랑한다.",
    "여성이 남성을 대체했을 때, 그들은 남성보다 훨씬 적은 돈을 벌게 되었고 이것이 진짜 문제였습니다.",
    "말 그대로 삶의 기초입니다.",
    "믿기 어렵겠지만, 러시아의 인민 예술가이자 가수이자 텔레비전 진행자인 나데즈다 밥키나는 70세입니다!",
    "미국에 도착한 누노는 제인 데일 해든 선생님으로 임명되었다.",
    "이 중재된 대화를 통해 스테파니아와 안드레아는 그들의 관계에 대해 반성하고 게임이 공개되면 더 큰 인식으로 게임을 재개할 수 있습니다.",
    "이러한 분석 결과는 최근의 글로벌 금융위기 기간에 잘 드러났듯이 국내외 금융 시장 간 연계성이 외환위기 이후 커진 가운데 해외 금융시장의 변동성이 국내 금융 시장에 미치는 파급효과도 보다 장기적이고 지속적으로 변화된 것을 의미한다."

]
inputs = tokenizer(texts_ko, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["en"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("KO→EN:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("\nKO→EN:")
for i, output in enumerate(outputs):
    print(f"{texts_ko[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


EN→KO:
The weather is very good today. → 오늘날 날씨는 매우 좋습니다.
I am studying machine learning. → 저는 기계학을 공부하고 있습니다.
Artificial intelligence is changing the world. → 인공지능은 세상을 변화시킨다.
Deep learning is a subset of machine learning. → 깊은 학습은 기계 학습의 하위 세트입니다.
Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day. → GM 판매자 인 Roberto는 매일 글을 쓰는 Ornella Muti와 미친 듯 사랑에 빠져 있습니다.
When women replaced men, they earned much less money than them, which was a real problem. → 여성들이 남자를 대신했을 때, 그들은 그들보다 훨씬 적은 돈을 벌었고, 이는 실제 문제였습니다.
It is literally the basis of life. → 그것은 문자 그대로 삶의 기초입니다.
It's hard to believe, but the People's Artist of Russia, singer and TV presenter Nadezhda Babkina is 70 years old! → 그것은 믿기 어렵지만, 러시아의 민간 아티스트, 노래와 TV 선생 Nadezhda Babkina는 70 년입니다!
Arriving in the USA, Nuno is assigned the teacher Jane Dayle Haddon as a &quot;tutor&quot;. → 미국에 도착하면서, Nuno는 교사 Jane Dayle Haddon에게 &quot;교사&quot;로 임명됩니다.
This mediated dialogue allows Stefania and Andrea to refle

In [20]:
import sacrebleu

# 영어 → 한국어 평가
references_ko = [
    "오늘 날씨는 매우 좋습니다.",
    "저는 머신러닝을 공부하고 있습니다.",
    "인공지능은 세상을 변화시키고 있습니다.",
    "딥러닝은 머신러닝의 하위 집합입니다.",
    "지엠 판매원 로베르토는 오르넬라 무티를 열렬히 사랑하며 매일 편지를 씁니다."
]

hypotheses_ko = [
    "오늘날 날씨는 매우 좋습니다.",
    "저는 기계 학습을 공부하고 있습니다.",
    "인공 지능이 세상을 바꾸고 있다.",
    "깊은 학습은 기계 학습의 하위 세트입니다.",
    "GM 판매자 인 로베로는 매일 글을 쓰는 오넬라 무티와 미친 사랑에 빠져 있습니다."
]

print("EN→KO chrF Score:")
chrf_ko = sacrebleu.corpus_chrf(hypotheses_ko, [references_ko])
print(chrf_ko)


# 한국어 → 영어 평가
references_en = [
    "The weather is very good today.",
    "I am studying machine learning.",
    "Artificial intelligence is changing the world.",
    "Deep learning is a subset of machine learning.",
    "Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day."
]

hypotheses_en = [
    "The weather is very good today.",
    "I am studying machine learning.",
    "Artificial intelligence changes the world.",
    "Deep learning is a subset of machine learning.",
    "Roberto, a Jim-seller, loves Ornella Muti, whose letters he writes every day."
]

print("\nKO→EN chrF Score:")
chrf_en = sacrebleu.corpus_chrf(hypotheses_en, [references_en])
print(chrf_en)

EN→KO chrF Score:
chrF2 = 31.88

KO→EN chrF Score:
chrF2 = 77.53
